In [2]:
import torch
import torch.nn as nn

class network(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.main = torch.nn.Sequential(
            nn.Linear(100, 100),
            nn.BatchNorm1d(100),
            nn.Linear(100,1),
            nn.Sigmoid()
        )
        
    def forward(self, inp):
#         for layer in self.main:
#             inp = layer(inp)
        print('forward')
        return self.main(inp)



In [5]:
from multiprocessing import Pool
# import torch.multiprocessing as mp
ref_net = network()
def f(x):
    net = network()
    net.load_state_dict(ref_net.state_dict())
    print('setup')
#     with torch.no_grad():
#         print(next(net.parameters()))
    out = net(x)
    print('output')
    return out.detach()

a = list(torch.randn(5,2,100))
# for i in range(2):
with Pool(5) as p:
    print(p.map(f, a))
fcn = torch.nn.Sequential(
            nn.Linear(100, 100),
            nn.BatchNorm1d(100),
            nn.Linear(100,1),
            nn.Sigmoid()
        )
b = fcn(torch.randn(2,100))
with Pool(5) as p:
    print(p.map(f, a))

with Pool(5) as p:
    print(p.map(f, a))

with Pool(5) as p:
    print(p.map(f, a))


setupsetupsetup
setup

setupforwardforward


forwardforward
forward


output
output
output
output
output
[tensor([[0.4567],
        [0.5829]]), tensor([[0.3078],
        [0.7254]]), tensor([[0.3757],
        [0.6612]]), tensor([[0.4358],
        [0.6033]]), tensor([[0.4414],
        [0.5979]])]
setupsetupsetup

setupforwardsetup
forward

forward

forward
forward



Process ForkPoolWorker-36:
Process ForkPoolWorker-40:
Process ForkPoolWorker-39:
Process ForkPoolWorker-38:
Process ForkPoolWorker-37:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._t

KeyboardInterrupt: 

In [ ]:
import torch 
from torch import multiprocessing,nn 
 
output_size = 13 
input_dim = output_size 
hidden_dim = 128 
layer_dim = 1 
 
lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=layer_dim, batch_first=True) 
torch.save(lstm.state_dict(),"plain.pt") 
 
def run(state): 
    model = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=layer_dim, batch_first=True) 
    model.load_state_dict(torch.load("plain.pt")) 
    x=model(state) 
    return x 
 
state=torch.rand(1,10,13) 
threads=[] 
for i in range(1): 
    p = multiprocessing.Process(target=run,args=(state,)) 
    threads.append(p) 
for i in threads: 
    i.start() 
for i in threads: 
    i.join() 
print("CHECKPOINT One") 
# run(state) 
fcn = torch.nn.Sequential(
            nn.Linear(100, 100),
            nn.BatchNorm1d(100),
            nn.Linear(100,1),
            nn.Sigmoid()
        )
a = fcn(torch.randn(2,100))
print("CHECKPOINT Two")
threads=[] 
for i in range(1): 
    p = multiprocessing.Process(target=run,args=(state,)) 
    threads.append(p) 
for i in threads: 
    i.start() 
for i in threads: 
    i.join()
print("CHECKPOINT Three")

CHECKPOINT One
CHECKPOINT Two


In [16]:

from setting import TASK

from antenna_selection.as_bb import ASBBenv as Environment, DefaultBranchingPolicy, solve_bb
if TASK == 'antenna_selection':
    from antenna_selection.observation import Observation, LinearObservation

elif TASK == 'single_cast_beamforming':
    from single_beamforming.observation import Observation, LinearObservation
    # from single_beamforming.acr_bb import ACRBBenv as Environment, DefaultBranchingPolicy, solve_bb

import torch
import torch.nn as nn
import numpy as np

from gnn_policy import GNNPolicy, GNNNodeSelectionPolicy
from tqdm import tqdm
import torch_geometric
import gzip
import pickle
from gnn_dataset import GraphNodeDataset, instance_generator
from pathlib import Path
from gnn_dataset import get_graph_from_obs
import shutil
import time
import os
from multiprocessing import Pool

from models.fcn_policy import FCNNodeSelectionLinearPolicy, FCNNodeDataset
from torch.utils.data import DataLoader

MAX_STEPS = 1000
    


class DataCollect(object):
    def __init__(self, observation_function=Observation, max_ant=None, policy='oracle',filepath=None, policy_type='gnn'):
        # env = Environment(observation_function=observation_function, epsilon=0.002)
        self.observation_function = observation_function
        self.max_ant = max_ant
        self.filepath = filepath
        self.branching_policy = DefaultBranchingPolicy()
        self.policy_type = policy_type
        self.node_select_policy = None


    def collect_data(self, instance_gen, num_instances=10, policy='oracle'):
        # self.node_select_policy = policy.copy()
        N, M = next(instance_gen).shape[1], next(instance_gen).shape[2]
        H = np.random.randn(num_instances, N, M) + 1j*np.random.randn(num_instances, N,M)    
        instances = np.stack((np.real(H), np.imag(H)), axis=1)
        

        arguments_oracle = list(zip(list(instances), [self.max_ant]*num_instances))
        print('starting first pool')
        with Pool(num_instances) as p:
            out_oracle = p.map(self.solve_bb_process, arguments_oracle)
            print('first pool ended')


        optimal_solution_list = [out_oracle[i][0] for i in range(len(out_oracle))]
        optimal_objective_list = [out_oracle[i][1] for i in range(len(out_oracle))]

        arguments_ml = list(zip(list(instances), optimal_solution_list, optimal_objective_list, range(num_instances), [policy]*num_instances))
        print('starting second pool')
        with Pool(num_instances) as p:
            out_ml = p.map(self.collect_data_instance, arguments_ml)
            # out_ml = p.map(self.dummy_collect_instance, arguments_ml)

            print('second pool ended')
        
        avg_oracle_steps = np.mean(np.array([out_oracle[i][2] for i in range(len(out_oracle))]))
        avg_oracle_time = np.mean(np.array([out_oracle[i][3] for i in range(len(out_oracle))]))
        avg_ml_time = np.mean(np.array([out_ml[i][0] for i in range(len(out_ml))]))
        avg_ml_ogap = np.mean(np.array([out_ml[i][1] for i in range(len(out_ml))]))
        avg_ml_steps = np.mean(np.array([out_ml[i][2] for i in range(len(out_ml))]))

        return avg_ml_time/avg_oracle_time, avg_ml_ogap, avg_ml_steps/avg_oracle_steps
        # return 0, 0, 0

    def dummy_collect_instance(self, arguments):
        instance, w_optimal, optimal_objective, file_count = arguments
        print('started collect instance {}'.format(file_count))
        import time
        time.sleep(1)
        print('ended collect instance {}'.format(file_count))


    def collect_data_instance(self, arguments):
        instance, w_optimal, optimal_objective, file_count, policy = arguments
        print('function {} started'.format(file_count))
        #TODO: do the following with parameters not filename
        env = Environment(observation_function=self.observation_function, epsilon=0.002)
        env.set_node_select_policy(node_select_policy_path=policy, policy_type=self.policy_type)
        
        env.reset(instance, max_ant=self.max_ant,  oracle_opt=w_optimal)
        branching_policy = DefaultBranchingPolicy()
        t1 = time.time()
        timestep = 0
        done = False
        time_taken = 0
        sum_label = 0
        node_counter = 0
        while timestep < MAX_STEPS and len(env.nodes)>0 and not done:
            print('data instance timestep', timestep, env.global_U, env.global_L)
            print('function {} now fathom'.format(file_count))
            
            env.fathom_nodes()
            if len(env.nodes) == 0:
                break
            node_id, node_feats, label = env.select_node()

            if len(env.nodes) == 0:
                break
            time_taken += time.time()-t1
            sum_label += label
            self.save_file((node_feats, label), file_count, node_counter)
            node_counter += 1
            t1 = time.time()
            print('function {} now prune'.format(file_count))

            prune_node = env.prune(node_feats)
            # prune_node = False
            if prune_node:
                env.delete_node(node_id)
                continue
            else:
                print('function {} now push children'.format(file_count))

                branching_var = branching_policy.select_variable(node_feats, env.action_set_indices)
                done = env.push_children(branching_var, node_id)
            timestep = timestep+1
        print('function {} now ended'.format(file_count))
        
        ml = np.linalg.norm(env.W_incumbent, 'fro')**2
        ogap = ((ml - optimal_objective)/optimal_objective)*100
        time_taken += time.time() - t1
        print('instance result', timestep, ogap, time_taken, sum_label, optimal_objective, ml)
        return timestep, ogap, time_taken

    def solve_bb_process(self, tup):
        instance, max_ant = tup
        return solve_bb(instance, max_ant)
        

    def save_file(self, sample, file_count, node_counter):
        print('file {}_{} save started'.format(file_count, node_counter))
        if self.filepath is not None:
            filename = self.filepath + f'sample_{file_count}_{node_counter}.pkl'
            with gzip.open(filename, 'wb') as f:
                pickle.dump(sample, f)
        print('file {} {} saved'.format(file_count, node_counter))


tensor([[ 0.9191,  0.5761, -0.8266, -1.2741,  0.6247,  0.2905, -0.6033,  0.1773,
          0.9678,  1.5657,  0.6728,  1.1733, -0.1721, -0.7883, -0.1769,  0.0442,
          1.0815, -1.6525, -1.2967,  0.8736,  0.1240,  0.3329, -0.1628, -0.4552,
         -1.9090, -0.3202, -0.6133, -0.1693, -0.3698,  0.0447, -1.2526,  0.8087,
          0.2611, -0.3090,  0.8964, -0.3075,  0.5231,  0.2576, -0.9545,  0.2401,
         -0.8211,  1.1077, -0.0126, -1.2956,  0.5740, -0.4767,  1.0674, -0.6908,
         -0.7485,  0.0574, -2.2930,  1.1796,  0.1569,  0.1387, -0.2047, -0.7179,
         -0.9344,  0.8584, -0.7811, -1.3700,  1.2836,  2.0756, -0.1297,  0.8812,
          0.1470,  0.5177,  0.3993, -1.7327,  0.2303,  0.0840,  0.8305, -0.3574,
         -0.8228,  1.2897, -0.6825, -0.8002, -0.2280, -0.1035,  0.6795,  0.7520,
         -0.0608,  1.2787,  0.3450,  0.2175, -0.5777,  0.8274,  0.9988, -0.1459,
          2.1954, -0.0731,  0.0051,  1.2742,  0.3184, -1.5703,  0.1134,  1.1136,
         -1.1635, -1.9181, -

In [ ]:
instance, w_optimal, optimal_objective, file_count, policy = arguments
        print('function {} started'.format(file_count))
        #TODO: do the following with parameters not filename
        env = Environment(observation_function=self.observation_function, epsilon=0.002)
        env.set_node_select_policy(node_select_policy_path=policy, policy_type=self.policy_type)
        
        env.reset(instance, max_ant=self.max_ant,  oracle_opt=w_optimal)
        branching_policy = DefaultBranchingPolicy()
        t1 = time.time()
        timestep = 0
        done = False
        time_taken = 0
        sum_label = 0
        node_counter = 0
        while timestep < MAX_STEPS and len(env.nodes)>0 and not done:
            print('data instance timestep', timestep, env.global_U, env.global_L)
            print('function {} now fathom'.format(file_count))
            
            env.fathom_nodes()
            if len(env.nodes) == 0:
                break
            node_id, node_feats, label = env.select_node()

            if len(env.nodes) == 0:
                break
            time_taken += time.time()-t1
            sum_label += label
            node_counter += 1
            t1 = time.time()
            print('function {} now prune'.format(file_count))

            prune_node = env.prune(node_feats)
            # prune_node = False
            if prune_node:
                env.delete_node(node_id)
                continue
            else:
                print('function {} now push children'.format(file_count))

                branching_var = branching_policy.select_variable(node_feats, env.action_set_indices)
                done = env.push_children(branching_var, node_id)
            timestep = timestep+1

In [8]:
a = list(torch.randn(5,2,100))